In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4' # nvidia-smi로 비어있는 gpu 확인하고 여기서 선택할것!


#from huggingface_hub import notebook_login

#notebook_login()

In [2]:
import numpy as np
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2
import pandas as pd

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [3]:
# 문장 예시
preds = [
    "이번 경진대회는 질의 응답 처리를 수행하는 AI 모델을 개발해야합니다.",
    "데이콘은 플랫폼입니다."
]

gts = [
    "이번 경진대회의 주제는 도배 하자 질의 응답 AI 모델 개발입니다.",
    "데이콘은 국내 최대의 AI 경진대회 플랫폼입니다."
]

In [4]:
# 샘플에 대한 Cosine Similarity 산식
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [5]:
sample_scores = []
for pred, gt in zip(preds, gts):
    # 생성된 답변 내용을 512 Embedding Vector로 변환
    pred_embed = model.encode(pred)
    gt_embed = model.encode(gt)
    
    sample_score = cosine_similarity(gt_embed, pred_embed)
    # Cosine Similarity Score가 0보다 작으면 0으로 간주
    sample_score = max(sample_score, 0)
    print('예측 : ', pred)
    print('정답 : ', gt)
    print('Cosine Similarity Score : ', sample_score)
    print('-'*20)
    sample_scores.append(sample_score)
print('전체 샘플의 Cosine Similarity Score 평균 : ', np.mean(sample_scores))

예측 :  이번 경진대회는 질의 응답 처리를 수행하는 AI 모델을 개발해야합니다.
정답 :  이번 경진대회의 주제는 도배 하자 질의 응답 AI 모델 개발입니다.
Cosine Similarity Score :  0.83436465
--------------------
예측 :  데이콘은 플랫폼입니다.
정답 :  데이콘은 국내 최대의 AI 경진대회 플랫폼입니다.
Cosine Similarity Score :  0.6281873
--------------------
전체 샘플의 Cosine Similarity Score 평균 :  0.731276


In [6]:
input_list = []


data = pd.read_csv("./bigdata/train.csv")
input_dict = {"utterance":[]}
for _, row in data.iterrows():
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            input_list.append({"question":row[q_col],"answer":row[a_col]})
            


In [7]:
def read_input(path):
    test_question = []
    test = pd.read_csv(path)
    for ut in test['질문']:
        test_question.append(ut)

    return test_question

In [8]:
test_q= read_input('./bigdata/test.csv')

In [9]:
gt_q_embed = []

for gt_q_dic in input_list:
    gt_q_embed.append({"question_embed":model.encode(gt_q_dic["question"]),"question":gt_q_dic["question"],"answer":gt_q_dic["answer"]})
    


In [16]:


def translate(text):
    max_sim_dic = {"question_embed":None,"question":None,"answer":None}
    max_sim = -100
    text_embed = model.encode(text)
    for gt_dic in gt_q_embed:
        tmp_sim = cosine_similarity(gt_dic["question_embed"], text_embed)
        if tmp_sim > max_sim:
            if "장점" or "단점" in text:
                if ("장점" in text and "단점" in gt_dic["question"]) or ("단점" in text and "장점" in gt_dic["question"]):
                    #print("there was an error")
                    continue
            max_sim = tmp_sim
            max_sim_dic = {"question_embed":gt,"question":gt_dic["question"],"answer":gt_dic["answer"]}
       
    
    return max_sim_dic["answer"]

In [17]:
def translate_framework(text):
    pred = ""
    dividers = ["또한,","그리고"]
    for divider in dividers:
        
        if divider in text:
            text1 = text.split(divider)[0].strip()
            text2 = text.split(divider)[1].strip()
            pred = translate(text1) +" "+divider+" "+ translate(text2)
            return pred
    else:
        pred = translate(text)
        return pred
    

In [18]:
preds = []
for i in range(len(test_q)):
    sr_text = test_q[i] # dialect
    tg_text = translate_framework(sr_text) # translate
    preds.append(tg_text)
    if i<300:
        print("source_text : ",sr_text)
        print("result :      ",tg_text)
        print()
        print()

source_text :  방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
result :       방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.


source_text :  도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
result :       해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상입니다. 도배지에 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.


source_text :  큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
result :       큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.


source_tex

In [13]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

(130, 512)

In [14]:
submit = pd.read_csv('./bigdata/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.026534,-0.020143,0.015142,0.017840,0.064733,0.019003,0.002009,-0.006164,-0.006306,...,-0.013746,-0.000601,-0.032005,-0.039729,0.001504,0.022239,0.030455,0.039992,0.001381,0.013259
1,TEST_001,-0.023421,-0.041167,0.039610,0.005828,0.052292,0.001335,-0.017209,0.012354,-0.010817,...,0.012853,-0.014981,0.007259,-0.051459,-0.043125,0.052939,0.016077,-0.038103,0.043188,-0.012774
2,TEST_002,0.027483,-0.047896,-0.024396,-0.000208,0.122577,-0.054817,-0.028307,-0.026099,0.043025,...,-0.034634,-0.017080,0.026628,-0.023708,0.022724,0.013046,-0.042113,-0.038604,-0.048141,0.092212
3,TEST_003,0.012937,0.010828,0.039439,-0.025302,-0.085114,0.036850,-0.075234,-0.015612,0.027546,...,0.002707,0.037181,0.056598,0.005570,0.001142,-0.007538,-0.032400,0.030189,-0.047903,0.015931
4,TEST_004,-0.043673,-0.049715,-0.022141,-0.005375,-0.022107,-0.002669,0.077968,0.026049,-0.039286,...,-0.046560,0.019589,-0.010394,0.026634,-0.024329,0.044780,-0.043111,0.015072,0.066883,-0.051414


In [15]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('./submit/dacon-v0_submit.csv', index=False)